In [39]:
from glob import glob
import json
import re
import yaml
import os
os.chdir('../')

In [33]:
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [38]:
coordinate_pathes = glob('../data/tweets/*.json')
len(coordinate_pathes)

43643

In [37]:
pathes = glob('../data/tweet_no_coordinates/*.json')
len(pathes)

37591

In [21]:
hashtags = ['bitcoin', 'blockchain', 'btc', 'cryptocurrency', 'market\svalue',
            'crypto','ethereum', 'fintech', 'coin', 'doge', 'ethereum', 'ripple', 'litecoin', 
            'cardano', 'monero', 'TRON', 'zcash', 'jaxx', 'copay', 'bitpay', 'exodus', 
            'mycelium', 'Bread\sWallet', 'trezor', 'ledger\snano', 'Silk\sRoad', 'darknet\smarket', 
            'dogecoin', 'ASIC\sMiner', 'Central\sLedger', 'Hashrate', 'ICO',         
           ]

In [22]:
'|'.join(hashtags)

'bitcoin|blockchain|btc|cryptocurrency|market\\svalue|crypto|ethereum|fintech|coin|doge|ethereum|ripple|litecoin|cardano|monero|TRON|zcash|jaxx|copay|bitpay|exodus|mycelium|Bread\\sWallet|trezor|ledger\\snano|Silk\\sRoad|darknet\\smarket|dogecoin|ASIC\\sMiner|Central\\sLedger|Hashrate|ICO'

In [23]:
bitcoin_pathes = []
for path in (pathes + coordinate_pathes):
    with open(path, 'r') as fp:
        data = json.load(fp)
        if re.search('|'.join(hashtags), json.dumps(data)):
            bitcoin_pathes.append(path)
        #print(data['entities']['hashtags'])


In [24]:
len(bitcoin_pathes)

374

In [16]:
for path in bitcoin_pathes[:6]:
    with open(path, 'r') as fp:
        data = json.load(fp)
data


{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Apr 27 07:30:29 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/9…',
    'expanded_url': 'https://twitter.com/i/web/status/989768732128886787',
    'indices': [107, 130],
    'url': 'https://t.co/9UztEJBlnB'}],
  'user_mentions': [{'id': 975165471388352512,
    'id_str': '975165471388352512',
    'indices': [44, 52],
    'name': "PlayerUnknown's Battleground Australia",
    'screen_name': 'PUBGaus'}]},
 'extended_tweet': {'display_text_range': [0, 288],
  'entities': {'hashtags': [],
   'symbols': [],
   'urls': [],
   'user_mentions': [{'id': 975165471388352512,
     'id_str': '975165471388352512',
     'indices': [44, 52],
     'name': "PlayerUnknown's Battleground Australia",
     'screen_name': 'PUBGaus'},
    {'id': 731770043969634304,
     'id_str': '731770043969634304',
     'indices': [106, 118],
     'name': 'Athletico',
     'screen_name': 'Athleti